In [1]:
from hdwallet import BIP141HDWallet
from hdwallet.symbols import FOXD
from hdwallet.cryptocurrencies import get_cryptocurrency
from src.rpc import RPC, get_address_utxos
from __data__ import credentials, mnemonic, settings, sighash


rpc = RPC(credentials['username'], credentials['password'], 8770)
sighash_types = sighash
language = settings['language']
password = settings['password']
phrase = mnemonic[-1]

In [7]:
# wallet = BIP44HDWallet(FOXD, get_cryptocurrency(FOXD))
# wallet.from_mnemonic(PHRASE, LANGUAGE, PASSWORD)
# wallet.dumps()['addresses']

In [12]:
path = [
    lambda a, b, c: f"m/44'/175'/{a}'/{b}/{c}"
]
wallet = BIP141HDWallet(FOXD, get_cryptocurrency(FOXD), path[0](0, 2, 1))
wallet.from_mnemonic(phrase, language, password)
wallet.dumps()['addresses']

{'p2pkh': 'FF9y6S2an9CjTkhrv9oGYU8YLWNfMMLEkZ',
 'p2sh': 'D7ZH91bXj6aXyxZJGz4kDmMASotf1761oE',
 'p2wpkh': 'fx1dve9qew0e6apy3wnwxwthmmkdv3lp7yq89t9g79',
 'p2wpkh_in_p2sh': 'DK1dNjCJuURNnrp8YnTjeb6wDsrVwpozyD',
 'p2wsh': 'fx1d8w2lvc9cwharslmgz2rwyrrgyma9u3exhtcvv3ac6w6q5tvxxh5qzxcxyh',
 'p2wsh_in_p2sh': 'DNWYfjS8TNwK1gQqDUMmozQRi6wNHDbNLV'}

In [25]:
rpc.getaddressbalance('FJWtzuU2HcJsCMt57NMQeZCfY9KgxLKrBj')

{'balance': 1000014705882, 'received': 1000014705882}

In [26]:
get_address_utxos(rpc, ['FJWtzuU2HcJsCMt57NMQeZCfY9KgxLKrBj', wallet.p2pkh_address()])

[{'address': 'FJWtzuU2HcJsCMt57NMQeZCfY9KgxLKrBj',
  'assetName': 'FOXD',
  'txid': '101e7c86b99a503b72fdcd1c89d88ed4d16cdb2305bbfd6641337cf6550a7235',
  'outputIndex': 1,
  'script': '76a9148b27be62181424c37ba5bfdfa061a3f800e50a3888ac',
  'satoshis': 1000000000000,
  'height': 1647207},
 {'address': 'FF9y6S2an9CjTkhrv9oGYU8YLWNfMMLEkZ',
  'assetName': 'FOXD',
  'txid': '1c018886dfcb2b858109e53145f044355f52b55a84f9224461533ddb1e6a7a3a',
  'outputIndex': 1,
  'script': '76a914664a0cb9f9d74248ba6e33977deecd647e1f100788ac',
  'satoshis': 500000000,
  'height': 1649303},
 {'address': 'FJWtzuU2HcJsCMt57NMQeZCfY9KgxLKrBj',
  'assetName': 'FOXD',
  'txid': 'a13f71d82e496cd1d4192345fd5040ad9bab5c68a7c9a5adc5599218b39de29d',
  'outputIndex': 0,
  'script': '76a9148b27be62181424c37ba5bfdfa061a3f800e50a3888ac',
  'satoshis': 14705882,
  'height': 1649308}]

In [20]:
1000000000000 / 1e8

10000.0

In [ ]:
from hashlib import new
def ripemd160(data): return new('ripemd160', data)
ripemd160(b'hello').hexdigest()

'108f07b8382412612c048d07d13f814118445acd'

In [6]:
from hdwallet.libs.bech32 import (
    decode
)
p2wsh = wallet.p2wsh_address()
